This code convert original data representation in new data representation as follow:

Original: samples => current values

This Representation: samples => current value concatenated with the difference between current values and previous one

E.G.

ORIGINAL
T0 = 2 4 5 1 4
T1 = 1 2 0 2 1
T2 = 2 3 4 0 2

NEW REPRESENTATION
T0 = 2  2  4  4  5  5  1  1  4  4
T1 = 1 -1  2 -2  0 -5  2  1  1 -3
T2 = 2  1  3  1  4  4  0 -2  2  1

In [1]:
import numpy as np
import pandas as pd

In [67]:
# ORIGINAL DATA
columns = ['w', 'q_chamber1_cilinda', 'q_chamber1_cilindb', 'q_chamber2_cilinda', 'q_chamber2_cilindb', 'q_pressureline_leftailc1', 'q_pressureline_leftailc2', 'q_tank_leftailc1', 'q_tank_leftailc2', 'f_cilinda', 'f_cilindb', 'x_cilinda', 'x_cilindb', 'p_chamber1_cilinda', 'p_chamber1_cilindb', 'p_chamber2_cilinda', 'p_chamber2_cilindb', 'p_pressureline_leftailc1', 'p_pressureline_leftailc2', 'p_tank_leftailc1', 'p_tank_leftailc2', 't', 'v_cilinda', 'v_cilindb', 'y_servo_valve']

# New column names to difference data
d_columns = ["d_"+c for c in columns]

# DATASET 1 - ALTITUDE MANEUVER
data_ds1_normal_original = pd.read_csv('../../dataset/data_representation_1/F16_DS1_normal.csv', names=columns)

In [ ]:
data_ds1_fault1_original = pd.read_csv('../../dataset/data_representation_1/F16_DS1_fault1_leakage.csv', names=columns)
data_ds1_fault2_original = pd.read_csv('../../dataset/data_representation_1/F16_DS1_fault2_viscousfriction.csv', names=columns)
data_ds1_fault3_original = pd.read_csv('../../dataset/data_representation_1/F16_DS1_fault3_compressibility.csv', names=columns)
data_ds1_fault4_original = pd.read_csv('../../dataset/data_representation_1/F16_DS1_fault4_fixedposition.csv', names=columns)

# DATASET 2 - ROLL MANEUVER + TORQUE SURFACE
data_ds2_normal_original = pd.read_csv('../../dataset/data_representation_1/F16_DS2_normal.csv', names=columns)
data_ds2_fault1_original = pd.read_csv('../../dataset/data_representation_1/F16_DS2_fault1_leakage.csv', names=columns)
data_ds2_fault2_original = pd.read_csv('../../dataset/data_representation_1/F16_DS2_fault2_viscousfriction.csv', names=columns)
data_ds2_fault3_original = pd.read_csv('../../dataset/data_representation_1/F16_DS2_fault3_compressibility.csv', names=columns)
data_ds2_fault4_original = pd.read_csv('../../dataset/data_representation_1/F16_DS2_fault4_fixedposition.csv', names=columns)

In [74]:
datasets = {'F16_DS1_normal': data_ds1_normal_original,
            'F16_DS1_fault1_leakage': data_ds1_fault1_original,
            'F16_DS1_fault2_viscousfriction': data_ds1_fault2_original,
            'F16_DS1_fault3_compressibility': data_ds1_fault3_original,
            'F16_DS1_fault4_fixedposition': data_ds1_fault4_original,
            'F16_DS2_normal': data_ds2_normal_original,
            'F16_DS2_fault1_leakage': data_ds2_fault1_original,
            'F16_DS2_fault2_viscousfriction': data_ds2_fault2_original,
            'F16_DS2_fault3_compressibility': data_ds2_fault3_original,
            'F16_DS2_fault4_fixedposition': data_ds2_fault4_original}

for k in datasets:

    v = datasets[k]

    # Create zeroed columns
    new_features = pd.DataFrame(np.zeros(v.shape), columns=d_columns)

    # Concatenate original columns with new columns
    ds = pd.concat([v, new_features], axis=1)

    for n, _ in v.iterrows():
        if n % 1400 == 0: # don't calculate the diff to the first register (new flight register)
            d = ds.iloc[n, :25] # copy values
        else:
            d = ds.iloc[n, :25] - ds.iloc[n-1, :25] # calculate the diff T(n) - T(n-1)

        d.index = d_columns
        ds.loc[n, d_columns] = list(d)

    ds.to_csv((str(k) + ".csv"), header=False, index=False)